<a href="https://colab.research.google.com/github/tomheston/fragility-metrics/blob/main/notebooks/PFI_RRI_RQ_RI_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# @title
# Exact UFI (Walter), PFI, RRI, RQ, and RI for a 2x2 table calculator
# v.11-NOV-2025
# Walter UFI uses Fisher's exact test (two-sided) with fixed margins and is an integer.
# PFI uses Pearson chi-square along a continuous fixed-margins path.
# Defaults alpha to 0.05.
#
# IF YOU USE THIS CALCULATOR PLEASE CITE:
#
# Heston, T. F. (2025). Fragility Metrics Toolkit (Version 2.1.0) [Software].
# Zenodo. https://doi.org/10.5281/zenodo.17254763
#
# © Thomas F. Heston 2025. CC-BY 4.0
#

import numpy as np
from scipy.stats import chi2_contingency, chi2, fisher_exact

ALPHA_DEFAULT = 0.05  # hard default

# ---------- Core utilities (Pearson χ² path for PFI/RRI/RQ/RI) ----------

def chi2_p_pearson(a, b, c, d):
    """
    Two-sided Pearson chi-square (no Yates). Returns (chi2_stat, p_value).
    """
    tbl = np.array([[a, b], [c, d]], dtype=float)
    if np.any(tbl < 0) or not np.isfinite(tbl.sum()) or tbl.sum() == 0:
        return (np.nan, np.nan)
    chi2_stat, p, dof, exp = chi2_contingency(tbl, correction=False)
    return float(chi2_stat), float(p)

def p_value_pearson(a, b, c, d):
    _, p = chi2_p_pearson(a, b, c, d)
    return p

def p_along_x(a, b, c, d, x):
    """
    Fixed-margins path for χ²: (a, b, c, d) -> (a+x, b-x, c-x, d+x)
    Continuous x permitted for PFI.
    """
    aa, bb, cc, dd = a + x, b - x, c - x, d + x
    if min(aa, bb, cc, dd) < 0:
        return np.nan
    return p_value_pearson(aa, bb, cc, dd)

# ---------- Fisher exact utilities (for Walter UFI) ----------

def fisher_p_two_sided(a, b, c, d):
    """
    Two-sided Fisher's exact p-value (hypergeometric, fixed margins).
    """
    tbl = np.array([[a, b], [c, d]], dtype=int)
    if np.any(tbl < 0) or tbl.sum() == 0:
        return np.nan
    _, p = fisher_exact(tbl, alternative="two-sided")
    return float(p)

def p_along_x_fisher(a, b, c, d, x):
    """
    Fixed-margins path for Fisher: integer x only.
    (a, b, c, d) -> (a+x, b-x, c-x, d+x)
    """
    aa, bb, cc, dd = a + x, b - x, c - x, d + x
    if min(aa, bb, cc, dd) < 0 or not np.isfinite(aa + bb + cc + dd):
        return np.nan
    return fisher_p_two_sided(aa, bb, cc, dd)

def compute_ufi_walter(a, b, c, d, alpha=ALPHA_DEFAULT):
    """
    Walter's UFI (hypergeometric):
    Minimum integer number of admissible swaps along the fixed-margins path
    needed to flip significance using two-sided Fisher's exact test.

    Returns (ufi_int, x_signed_int, p_at_flip) or (None, None, None) if no flip feasible.
    """
    a, b, c, d = map(int, (a, b, c, d))
    if min(a, b, c, d) < 0:
        raise ValueError("All cells must be >= 0.")

    # bounds for integer x along (a+x, b-x, c-x, d+x)
    x_min = -min(a, d)         # <= 0
    x_max =  min(b, c)         # >= 0

    p0 = fisher_p_two_sided(a, b, c, d)
    if not np.isfinite(p0):
        return (None, None, None)
    sig0 = (p0 <= alpha)

    # search outward in integer steps m = 1,2,... for either direction
    max_m = int(max(-x_min, x_max))
    flip_m = None
    flip_sign = None
    flip_p = None

    for m in range(1, max_m + 1):
        # try +m
        if m <= x_max:
            p_plus = p_along_x_fisher(a, b, c, d, m)
            if np.isfinite(p_plus):
                sig_plus = (p_plus <= alpha)
                if sig_plus != sig0:
                    flip_m, flip_sign, flip_p = m, +1, p_plus
                    break
        # try -m
        if m <= -x_min:
            p_minus = p_along_x_fisher(a, b, c, d, -m)
            if np.isfinite(p_minus):
                sig_minus = (p_minus <= alpha)
                if sig_minus != sig0:
                    flip_m, flip_sign, flip_p = m, -1, p_minus
                    break

    if flip_m is None:
        return (None, None, None)

    return (int(flip_m), int(flip_sign * flip_m), float(flip_p))

# ---------- Combined computation ----------

def compute_ufi_pfi(a, b, c, d, alpha=ALPHA_DEFAULT, tol=1e-10, max_iter=200):
    """
    Computes:
      - Walter's UFI (integer, Fisher two-sided, fixed margins)
      - Continuous χ²-based displacement x* to alpha boundary (for PFI)
      - PFI = |4x*|/n (decimal and percent)
      - RRI, RQ (p-value independent)
      - RI (Robustness Index) via χ² scaling

    Returns a dict with all fields.
    """
    a, b, c, d = map(float, (a, b, c, d))
    n = a + b + c + d
    if n <= 0:
        raise ValueError("Total n must be > 0.")
    if min(a, b, c, d) < 0:
        raise ValueError("All cells must be >= 0.")

    # RRI and RQ (p-value independent; χ² formulation consistent with fixed margins)
    rri = abs(a * d - b * c) / n
    rq  = (4.0 * rri) / n

    # Feasible x to keep all cells >= 0 along χ² path
    x_min = -min(a, d)      # left bound (<= 0)
    x_max =  min(b, c)      # right bound (>= 0)

    # Original χ², p and state
    chi2_0, p0 = chi2_p_pearson(a, b, c, d)
    if not np.isfinite(p0):
        raise ValueError("Invalid table for chi-square.")
    sig0 = (p0 <= alpha)

    # Independence location along path
    r1, r2 = a + b, c + d
    c1, c2 = a + c, b + d
    a_exp = r1 * c1 / n
    x_ind = a_exp - a  # displacement to independence

    # Target function for χ²-path PFI
    def f(x):
        return p_along_x(a, b, c, d, x) - alpha

    # Choose search interval toward flip (χ² path)
    if sig0:  # raise p toward independence
        lo, hi = (0.0, x_ind) if x_ind >= 0 else (x_ind, 0.0)
    else:     # lower p away from independence
        if x_ind >= 0:
            lo, hi = (0.0, x_min)  # x_min <= 0
        else:
            lo, hi = (0.0, x_max)  # x_max >= 0

    # Order, clip to feasibility
    if lo > hi:
        lo, hi = hi, lo
    lo = max(lo, x_min)
    hi = min(hi, x_max)

    x_star = None
    p_flip = None
    UFI_continuous = None
    PFI_decimal = None
    PFI_percent = None

    if lo != hi:
        # Bracket sign change; grid if needed
        flo, fhi = f(lo), f(hi)
        if not (np.isfinite(flo) and np.isfinite(fhi)) or flo * fhi > 0:
            GRID = 400
            xs = np.linspace(lo, hi, GRID + 1)
            fs = np.array([f(x) for x in xs])
            found = False
            for i in range(GRID):
                if np.isfinite(fs[i]) and np.isfinite(fs[i + 1]) and fs[i] * fs[i + 1] <= 0:
                    lo, hi = xs[i], xs[i + 1]
                    flo, fhi = fs[i], fs[i + 1]
                    found = True
                    break
            if not found:
                # No χ² flip along feasible path
                x_star = None
            else:
                # Bisection
                for _ in range(max_iter):
                    mid = 0.5 * (lo + hi)
                    fmid = f(mid)
                    if not np.isfinite(fmid):
                        mid = np.nextafter(mid, hi)
                        fmid = f(mid)
                        if not np.isfinite(fmid):
                            break
                    if abs(hi - lo) < tol:
                        x_star = mid
                        break
                    if flo == 0:
                        x_star = lo
                        break
                    if fhi == 0:
                        x_star = hi
                        break
                    if flo * fmid <= 0:
                        hi, fhi = mid, fmid
                    else:
                        lo, flo = mid, fmid
                else:
                    x_star = 0.5 * (lo + hi)
        else:
            # Bisection directly
            for _ in range(max_iter):
                mid = 0.5 * (lo + hi)
                fmid = f(mid)
                if not np.isfinite(fmid):
                    mid = np.nextafter(mid, hi)
                    fmid = f(mid)
                    if not np.isfinite(fmid):
                        break
                if abs(hi - lo) < tol:
                    x_star = mid
                    break
                if flo == 0:
                    x_star = lo
                    break
                if fhi == 0:
                    x_star = hi
                    break
                if flo * fmid <= 0:
                    hi, fhi = mid, fmid
                else:
                    lo, flo = mid, fmid
            else:
                x_star = 0.5 * (lo + hi)

    if x_star is not None:
        p_flip = p_along_x(a, b, c, d, x_star)
        UFI_continuous = abs(x_star)
        PFI_decimal = (4.0 * UFI_continuous) / n
        PFI_percent = 100.0 * PFI_decimal

    # ---------- Robustness Index (RI) via χ² scaling ----------
    chi2_crit = chi2.ppf(1.0 - alpha, df=1)
    if np.isfinite(chi2_0) and chi2_0 > 0:
        if p0 > alpha:
            # non-significant: multiply all cells by k until chi2(k) >= chi2_crit
            RI = chi2_crit / chi2_0
            RI_mode = "multiply cells by k"
        else:
            # significant: divide all cells by k until chi2(k) < chi2_crit
            RI = chi2_0 / chi2_crit
            RI_mode = "divide cells by k"
        if RI <= 1:  # numerical guard; enforce k > 1
            RI = 1.0 + 0.0
    else:
        RI = None
        RI_mode = None

    # ---------- Walter UFI (integer, Fisher two-sided, fixed margins) ----------
    ufi_walter, x_walter, p_walter = compute_ufi_walter(int(a), int(b), int(c), int(d), alpha=alpha)

    return {
        "inputs": {"a": a, "b": b, "c": c, "d": d},
        "alpha": alpha,
        "p_original": p0,
        # χ² continuous-path outputs (PFI-related)
        "x_signed_continuous": x_star,
        "UFI_continuous": UFI_continuous,    # |x*| from χ² path (NOT Walter)
        "PFI_decimal": PFI_decimal,
        "PFI_percent": PFI_percent,
        "p_at_flip_continuous": p_flip,
        # Walter UFI (correct integer, Fisher two-sided, fixed margins)
        "UFI_Walter": ufi_walter,
        "x_Walter": x_walter,
        "p_at_flip_Walter": p_walter,
        # p-value-independent metrics
        "RRI": rri,
        "RQ": rq,
        # Robustness Index via χ² scaling
        "RI": RI,
        "RI_mode": RI_mode
    }

# ---------- Single-run interactive ----------

def _fmt(x, fmt):
    return "None" if x is None else (fmt % x)

def run_once():
    print("Enter 2x2 counts (nonnegative). Press Enter to accept default in brackets.")
    def read_float(prompt, default=None):
        s = input(prompt).strip()
        if s == "" and default is not None:
            return float(default)
        return float(s)

    a = read_float("a [30]: ", 30)
    b = read_float("b [55]: ", 55)
    c = read_float("c [17]: ", 17)
    d = read_float("d [20]: ", 20)

    alpha = ALPHA_DEFAULT

    res = compute_ufi_pfi(a, b, c, d, alpha=alpha)

    print("\n--- Results ---")
    print(f"Inputs: a={res['inputs']['a']:.6g}, b={res['inputs']['b']:.6g}, c={res['inputs']['c']:.6g}, d={res['inputs']['d']:.6g}")
    print(f"Alpha: {res['alpha']}")
    # Fisher/Walter outputs
    print("Walter UFI (integer):",      _fmt(res['UFI_Walter'], "%d"))
    print("Walter x (signed int):",     _fmt(res['x_Walter'], "%d"))
    print("Fisher p-value at flip:",    _fmt(res['p_at_flip_Walter'], "%.10f"))
    # χ² continuous-path outputs
    print("Original p-value (Pearson):",      _fmt(res['p_original'], "%.10f"))
    print("x* (signed, χ² path):",            _fmt(res['x_signed_continuous'], "%.10f"))
    print("UFI_continuous (|x*|, χ² path):",  _fmt(res['UFI_continuous'], "%.10f"))
    print("PFI (decimal):",                   _fmt(res['PFI_decimal'], "%.10f"))
    print("PFI (%):",                         _fmt(res['PFI_percent'], "%.6f"))
    # p-independent and RI
    print("RRI:",                              _fmt(res['RRI'], "%.10f"))
    print("RQ:",                               _fmt(res['RQ'], "%.10f"))
    print("p-value at flip (χ² path):",        _fmt(res['p_at_flip_continuous'], "%.10f"))
    print("RI (k):",                           _fmt(res['RI'], "%.10f"))
    print("RI mode:",                          res['RI_mode'] if res['RI_mode'] is not None else "None")
    print("----------------")

if __name__ == "__main__":
    run_once()


Enter 2x2 counts (nonnegative). Press Enter to accept default in brackets.
a [30]: 527
b [55]: 16250
c [17]: 2235
d [20]: 28855

--- Results ---
Inputs: a=527, b=16250, c=2235, d=28855
Alpha: 0.05
Walter UFI (integer): 394
Walter x (signed int): 394
Fisher p-value at flip: 0.0535070429
Original p-value (Pearson): 0.0000000000
x* (signed, χ² path): 393.3515019837
UFI_continuous (|x*|, χ² path): 393.3515019837
PFI (decimal): 0.0328703701
PFI (%): 3.287037
RRI: 441.0588714563
RQ: 0.0368570306
p-value at flip (χ² path): 0.0500000000
RI (k): 85.4716685739
RI mode: divide cells by k
----------------
